In [37]:
from pixell import powspec
import math
import numpy as np
import sys, os
# add the parent dir in the python path                                                                                                                                                  
sys.path.append(os.path.dirname(os.getcwd()))                                                                                                                                            
import param as p  

In [1]:
cmb_sets = [0]
print(type(cmb_sets))

<class 'list'>


In [24]:
for cmb_set in cmb_sets:
    print(cmb_set)

0


In [25]:
lens_ps = '../inputPs/cosmo2017_10K_acc3_lenspotentialCls.dat'

In [26]:
ps = powspec.read_camb_full_lens(lens_ps)

In [19]:
sim_nums = [2, 3, 4]
sim_jobs = [1, 2]

nsim_per_job = np.ceil(len(sim_nums)/len(sim_jobs))

In [41]:
sim_num_chuncks = np.array_split(sim_nums, nsim_per_job)
qos = 'regular'
time = '00:30:00'
OMP_NUM_THREADS = 4
n_tasks_per_node = nsim_per_job * OMP_NUM_THREADS
job_path = p.repodir + 'jobs/'
sim_path = p.repodir + 'src'

In [45]:
for i, sim_job in enumerate(sim_jobs):
    f = open('../jobs/sim_job_%s' %sim_job, 'w')
    f.write('#!/bin/bash\n')
    f.write('#SBATCH --qos %s\n' %qos)
    f.write('#SBATCH --constraint=haswell\n')
    f.write('#SBATCH -N 1\n')
    f.write('#SBATCH -t %s\n' %time)
    f.write('#SBATCH --ntasks-per-node=%s\n' %n_tasks_per_node)
    f.write('#SBATCH --license=SCRATCH\n')
    f.write('#SBATCH --output=%sslurmjob.log.%d\n' %(job_path, sim_job))
    f.write('\n')
    f.write('cd %s\n\n' %sim_path)
    for sim_num in sim_num_chuncks[i]:
        print('%s: %s'%(sim_job,sim_num))    
        f.write('OMP_NUM_THREADS=%s python CMBLensed_sim.py --sim_num=%s\n' %(OMP_NUM_THREADS, sim_num))
    f.write('wait\n')
    f.close()

1: 2
1: 3
2: 4
